# RAG with LlamaIndex

### Install the required libraries

In [201]:
!pip install llama-index --quiet
!pip install pypdf --quiet
!pip install singlestoredb --quiet
!pip install llama-index-vector-stores-singlestoredb --quiet
!pip install llama-index-llms-openai --quiet

### Import required libraries

In [202]:
import os
from pathlib import Path  # Path is used for cross-platform file path handling
import requests
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores.singlestoredb import SingleStoreVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

### Set your API keys and environment variables

In [212]:
import os
os.environ["OPENAI_API_KEY"] = "Add Your OpenAI API Key"
os.environ["SINGLESTOREDB_URL"] = "admin:password@host_url:3306/database_name"

### Download and save the PDF

In [213]:
def download_pdf(url, output_path):
    """
    Downloads a PDF file from the given URL and saves it to the specified path
    """
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"PDF downloaded successfully to {output_path}")
    else:
        raise Exception(f"Failed to download PDF. Status code: {response.status_code}")

In [214]:
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

In [215]:
pdf_url = "https://unctad.org/system/files/official-document/wesp2023_en.pdf"
pdf_path = data_dir / "economic_report.pdf"
download_pdf(pdf_url, pdf_path)

PDF downloaded successfully to data/economic_report.pdf


### Initialize SingleStore vector store

In [216]:
from llama_index.vector_stores.singlestoredb import SingleStoreVectorStore
import os

vector_store = SingleStoreVectorStore(
    table_name="laxpav1",
    content_field="content",
    metadata_field="metadata",
    vector_field="vector",
    timeout=30,
)

### mention the embedding model and the required settings

In [223]:
embed_model = OpenAIEmbedding()
llm = OpenAI(temperature=0.1, model="gpt-4")

### Load and process the document

In [224]:
documents = SimpleDirectoryReader(input_dir=str(data_dir)).load_data()

In [225]:
documents = SimpleDirectoryReader(input_dir=str(data_dir)).load_data()
print(f"Number of documents loaded: {len(documents)}")
print(f"First document content preview: {documents[0].text[:200] if documents else 'No content'}")

Number of documents loaded: 178
First document content preview: 2023
World Economic Situation 
and Prospects


### Create and persist the index

In [226]:
index = VectorStoreIndex.from_documents(
    documents,
    vector_store=vector_store,
)

### Function to query the index

In [227]:
def query_document(query_text):
    """
    Queries the document using the provided text
    Returns the response from the model
    """
    query_engine = index.as_query_engine()
    response = query_engine.query(query_text)
    return response

### Example usage - Chat with PDF with your queries

In [278]:
if __name__ == "__main__":
    # Example queries
    queries = [
        "What are the main economic challenges discussed in the report?",
        "What are the key recommendations for policy makers?",
        "What about the Microchip supply?",
        "The microchip supply shortage is predicted to rise or fall down?",
        "What is Ireland's GDP growth shown in the report?",
        "How much is the clean energy investment?",
        "What was the job loss that happened in the United States that is stated in the report?",
    ]
    
    for query in queries:
        print(f"\nQuery: {query}")
        response = query_document(query)
        print(f"Response: {response}")


Query: What are the main economic challenges discussed in the report?
Response: The main economic challenges discussed in the report include tight labor markets in developed economies, the world being off track to eliminate extreme poverty, global trade facing significant challenges, volatile commodity markets, risks of overtightening monetary policies, revisiting inflation targets, the need for effective coordination between monetary and fiscal policies, better international partnerships for responding to ongoing shocks, economic crisis and output gaps, output losses from the pandemic, weak employment recovery, higher risks to food security, rising debt vulnerabilities in developing countries, and the lingering effects of the pandemic.

Query: What are the key recommendations for policy makers?
Response: Policy makers are advised to strike a balance between emergency spending to protect vulnerable populations and development spending to enhance economic productivity. They should prio